# SQL en PySpark


## 1. Instanciar una app de Spark
## 2. Importar ```SQLContext``` y ```Row``` de ```pyspark.sql```
## 3. Instanciar ```SQLContext``` al contexto de Spark

In [1]:
import pyspark
sc = pyspark.SparkContext(appName="ufoisrael")
sc.setLogLevel("WARN")

from pyspark.sql import SQLContext
from pyspark.sql import Row
sql = SQLContext(sc)

## Cargar las siguientes funciones de utileria 😉

In [2]:
import csv
import string
exclude = set(string.punctuation)

def parseLine(line):
    csvdata = csv.reader([line], delimiter='\t', quotechar='"')
    for fields in csvdata:
        try:
            summary = ''.join(ch for ch in fields[5].lower() if ch not in exclude)
            return {"City": fields[2], "Summary" : summary}
        except:
            return { "City" : "City", "Summary": "Summary"}
        
        
        
        
exclude = set(string.punctuation)
colors = ('silver','gold','white','black','green','purple','orange','yellow','red','blue')
directions =('north','south','east','west')

def parseLineExtractColorAndDirection(line):
    csvdata = csv.reader([line], delimiter='\t', quotechar='"')
    for fields in csvdata:
        try:
            summary = ''.join(ch for ch in fields[5].lower() if ch not in exclude)
            colorcounts = [1 if summary.find(str(c)) != -1 else 0 for c in colors]
            if (sum(colorcounts)==0):
                color='none'
            elif (sum(colorcounts)>1):
                color='multiple'
            else:
                color = colors[colorcounts.index(1)]
            directioncounts = [1 if summary.find(str(d)) != -1 else 0 for d in directions]
            if (sum(directioncounts)==0):
                direction='none'
            elif (sum(directioncounts)>1):
                direction='multiple'
            else:
                direction = directions[directioncounts.index(1)]

            return { "City": fields[2], "Color" : color,  "Direction": direction, "Summary" : summary}
        except:
            return { "City" : "City"  }
        
        
def toRow(dic):
    return Row(City=dic['City'],Color=dic['Color'],Direction=dic['Direction'],Summary=dic['Summary'])

## Obtener el dataset UFO_awesome

In [ ]:
!wget https://github.com/johnmyleswhite/ML_for_Hackers/blob/76410666caf8b0e725c07d3210aaacea66b347fb/01-Introduction/code/data/ufo/ufo_awesome.tsv?raw=true

Crear un RDD a partir del archivo TSV

In [3]:
data = sc.textFile('ufo_awesome.tsv')


['19951009\t19951009\t Iowa City, IA\t\t\tMan repts. witnessing &quot;flash, followed by a classic UFO, w/ a tailfin at back.&quot; Red color on top half of tailfin. Became triangular.']

Inspeccionar la cabecera del dataset

In [3]:
data.take(1)

['19951009\t19951009\t Iowa City, IA\t\t\tMan repts. witnessing &quot;flash, followed by a classic UFO, w/ a tailfin at back.&quot; Red color on top half of tailfin. Became triangular.']

Aplicar la función *parseLine* a cada línea del dataset y filtrar los elementos que no contienen ciudad registrada

In [4]:
ufo_data = data.map(parseLine).filter(lambda ufo: ufo['City'] != 'City')
ufo_data.take(5)


[{'City': ' Iowa City, IA',
  'Summary': 'man repts witnessing quotflash followed by a classic ufo w a tailfin at backquot red color on top half of tailfin became triangular'},
 {'City': ' Milwaukee, WI',
  'Summary': 'man  on hwy 43 sw of milwaukee sees large bright blue light streak by his car descend turn cross road ahead strobe bizarre'},
 {'City': ' Shelton, WA',
  'Summary': 'telephoned reportca woman visiting daughter witness discs and triangular ships over squaxin island in puget sound dramatic  written report with illustrations submitted to nuforc'},
 {'City': ' Columbia, MO',
  'Summary': 'man repts sonaposs bizarre sighting of small humanoid creature in back yard  reptd in acteon journal st louis ufo newsletter'},
 {'City': ' Seattle, WA',
  'Summary': 'anonymous caller repts sighting 4 ufoaposs in nne sky 45 deg above horizon  no other facts reptd  no return tel '}]

Crear tres RDDs donde almacenemos:

1. Resumen de cada avistamiento (map())
2. Todas las palabras contenidas en los resumenes  (flatMap().map())
3. Orientación (puntos cardinales) del avistamiento según las palabras filtradas en el RDD de palabras (reduceByKey().filter())

In [5]:
ufo_summaries = ufo_data.map( lambda ufo: ufo['Summary'])

ufo_words = ufo_summaries.flatMap(lambda summary: summary.split(' ')).map(lambda word: (word,1))

ufo_direction = ufo_words.reduceByKey(lambda a,b: a+b).filter( lambda tup: tup[0] in ('north','south','east','west'))

ufo_direction.collect()


[('east', 17893), ('north', 19565), ('west', 17868), ('south', 17955)]

Procesar el color y dirección del RDD original al aplicat la función ```parseLineExtractColorAndDirection``` en aquellas líneas que tienen registro de ciudad    ```(map().filter())```

In [6]:
ufo_data = data.map(parseLineExtractColorAndDirection).filter(lambda ufo: ufo['City'] != 'City')
ufo_data.take(5)


[{'City': ' Iowa City, IA',
  'Color': 'red',
  'Direction': 'none',
  'Summary': 'man repts witnessing quotflash followed by a classic ufo w a tailfin at backquot red color on top half of tailfin became triangular'},
 {'City': ' Milwaukee, WI',
  'Color': 'blue',
  'Direction': 'none',
  'Summary': 'man  on hwy 43 sw of milwaukee sees large bright blue light streak by his car descend turn cross road ahead strobe bizarre'},
 {'City': ' Shelton, WA',
  'Color': 'none',
  'Direction': 'none',
  'Summary': 'telephoned reportca woman visiting daughter witness discs and triangular ships over squaxin island in puget sound dramatic  written report with illustrations submitted to nuforc'},
 {'City': ' Columbia, MO',
  'Color': 'none',
  'Direction': 'none',
  'Summary': 'man repts sonaposs bizarre sighting of small humanoid creature in back yard  reptd in acteon journal st louis ufo newsletter'},
 {'City': ' Seattle, WA',
  'Color': 'none',
  'Direction': 'none',
  'Summary': 'anonymous caller

Repetir el paso anterior, pero exportar el RDD a un DF

In [7]:
ufo_data = data.map(parseLineExtractColorAndDirection).filter(lambda d: d['City'] != 'City')
ufo_dataframe = ufo_data.map(toRow).toDF()
ufo_dataframe.show(15)

+--------------------+-----+---------+--------------------+
|                City|Color|Direction|             Summary|
+--------------------+-----+---------+--------------------+
|       Iowa City, IA|  red|     none|man repts witness...|
|       Milwaukee, WI| blue|     none|man  on hwy 43 sw...|
|         Shelton, WA| none|     none|telephoned report...|
|        Columbia, MO| none|     none|man repts sonapos...|
|         Seattle, WA| none|     none|anonymous caller ...|
| Brunswick County...|white|     none|sheriffaposs offi...|
|           Fargo, ND|  red|     none|female student w ...|
|       Las Vegas, NV|  red|     none|man repts bright ...|
|          Morton, WA| none|     none|woman reports 2 c...|
|         Redmond, WA|white|     none|young man w 2 cow...|
|          Renton, WA| none|     none|man repts seeing ...|
|     Springfield, IL| none|     none|man amp mother re...|
|          Sharon, MA| none|     none|man witnessed quo...|
|         Laporte, WA|  red|     none|se

Exportar el DF a DF de Pandas

In [8]:
ufo_pandas = ufo_dataframe.toPandas()

In [19]:
ufo_pandas.head(5)

,City,Color,Direction,Summary
0,"Iowa City, IA",red,none,man repts witnessing quotflash followed by a c...
1,"Milwaukee, WI",blue,none,man on hwy 43 sw of milwaukee sees large brig...
2,"Shelton, WA",none,none,telephoned reportca woman visiting daughter wi...
3,"Columbia, MO",none,none,man repts sonaposs bizarre sighting of small h...
4,"Seattle, WA",none,none,anonymous caller repts sighting 4 ufoaposs in ...


Inspeccionar el esquma del DF

In [9]:
ufo_dataframe.printSchema()

root
 |-- City: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Direction: string (nullable = true)
 |-- Summary: string (nullable = true)



In [17]:
%timeit ufo_dataframe.groupBy(ufo_dataframe.Color) \
              .count() \
              .toPandas() \
              .sort_values('count')

1 loop, best of 3: 9.28 s per loop


In [18]:
ufo_dataframe.registerTempTable('ufo')
%timeit sql.sql("SELECT Color, count(*) as ColorCount FROM ufo GROUP BY Color ORDER BY ColorCount").toPandas()

1 loop, best of 3: 9.83 s per loop


In [19]:
sc.stop()